# 1. Import required modules and define function

In [1]:
import pandas as pd

import os, glob, sys

# define function for splitting excel
def split_channles_for_excel (data_df_fname, 
                              data_folder,
                              #num_of_channel=4, 
                              info_col_index=0, 
                              label_col_index=2, 
                              calculate_ratio=False,
                              ratio_col_name=None,
                              labelmark = 'channel_index', 
                              save_excel=True):
    
    # make subfolder
    sub_fdname = os.path.join(data_folder, 'Split_files')
    if not os.path.exists(sub_fdname):
        os.mkdir(sub_fdname)
    
    # load df with sheet index non-specified
    data_df = pd.read_excel(data_df_fname, None)
    # process each sheet
    for sheet_name, sheet_df in data_df.items():
        print (f'-- Split sheet {sheet_name} for the excel containing ', end='')
        
        # get channel info: channel must be annotated as format similar to "1_blue", "2_red", etc
        info_col = sheet_df[sheet_df.columns[info_col_index]].to_list()
        channel_list = []
        for _info in info_col[:10]:
            _info = str(_info)
            if len(_info.split('_'))==2 and _info.split('_')[0].isnumeric():
                channel_list.append(_info.split('_')[1]) 
        print (f'{len(channel_list)}-channels data.')
        
        # calculate 'MN/PN ratio [(i1-i3)/(i2-i3)]' if necessary 
        if calculate_ratio and not isinstance(ratio_col_name,type(None)):
            for i in range(len(sheet_df)):
                if i%3 == 0 and ratio_col_name in sheet_df.columns:
                    sheet_df.loc[i,'MN/PN'] = ((sheet_df.iloc[i][ratio_col_name]-sheet_df.iloc[i+2][ratio_col_name])/
                                    (sheet_df.iloc[i+1][ratio_col_name]-sheet_df.iloc[i+2][ratio_col_name]))
                else:
                    print ('-- data format is not correct; skip MN/PN ratio calculation.')
                 
                    
        # get label column to split by the channel 'landmark'
        #ch_index_list = []
        if len(channel_list)>0:
            for _i, _ch_name in enumerate(channel_list):
                if labelmark == 'channel_index':
                    ch_index = '_C_' + f'{_i+1}'
                    #ch_index_list.append('ch_index)
                elif labelmark == 'laser_index':
                    ch_index = 'w' + f'{_i+1}'
                    #ch_index_list.append('ch_index)
                else:
                    print ('-- Channel annotation incorrect; skip the excel.')
                    return None
            
                ch_df_index = sheet_df[sheet_df.columns[label_col_index]].str.contains(ch_index)
                ch_df = sheet_df[ch_df_index]
                ch_df_measure = ch_df[ch_df.columns[1:]]
            
                if save_excel:
                    ch_fname = data_df_fname.split('.xlsx')[0] + f'_{sheet_name}_{_ch_name}_analyzed.xlsx'
                    ch_fname_new = ch_fname.replace(data_folder,sub_fdname)
                    ch_df_measure.to_excel(ch_fname_new, index =False)
                    
        else:
            print ('-- Channel annotation incorrect; skip the excel.')
                
    return None

## 2. Process all excels  

In [2]:
# replace the path (contating the excels) inside the "     " for different analysis
data_folder = r"D:\Analyzed_CellBio\Stam\Revision_exps_p2\MN_20220803"

data_excel_f_list = [_f for _f in glob.glob(os.path.join(data_folder,"*")) if 'xlsx' in _f and 'analyzed' not in _f]
print (f'-- Found {len(data_excel_f_list)} excel files in the folder.')

-- Found 4 excel files in the folder.


In [3]:
# run function below

for _f in data_excel_f_list[:]:
    print (f'-- Process {_f}.')
    split_excel = split_channles_for_excel (_f, data_folder,calculate_ratio=True)
    print (f'-----------------------------------------------------')
    print (f'-----------------------------------------------------')

-- Process D:\Analyzed_CellBio\Stam\Revision_exps_p2\MN_20220803\220623IF_220617exp_U2OS_control.xlsx.
-- Split sheet Sheet1 for the excel containing 4-channels data.
-----------------------------------------------------
-----------------------------------------------------
-- Process D:\Analyzed_CellBio\Stam\Revision_exps_p2\MN_20220803\220623IF_220620exp_U2OS_control.xlsx.
-- Split sheet Sheet1 for the excel containing 4-channels data.
-----------------------------------------------------
-----------------------------------------------------
-- Process D:\Analyzed_CellBio\Stam\Revision_exps_p2\MN_20220803\220623IF_220621exp_control_asynchr.xlsx.
-- Split sheet Sheet1 for the excel containing 4-channels data.
-----------------------------------------------------
-----------------------------------------------------
-- Process D:\Analyzed_CellBio\Stam\Revision_exps_p2\MN_20220803\220623IF_220622exp_control_asynchr.xlsx.
-- Split sheet Sheet1 for the excel containing 4-channels data.
--